Александр, здравствуйте! Ниже вопросы, которые у меня возникли в ходе выполнения ДЗ. Подскажите, пожалуйста, как бороться с такими явлениями? (можно в комментариях к ДЗ, либо на каком-нибудь семинаре)

In [ ]:
# Импорт необходимых библиотек

from lxml import html
import requests
import json
import pandas as pd
import re

In [ ]:
url = 'https://new.russwimming.ru/data/list/treneri_specialisti.html'
resp = requests.get(url=url, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'})
tree = html.fromstring(resp.content)

**Вопрос 1.**<br>
Код html страницы с таблицей, где указан тренерский состав, на сайте Всероссийской федерации плавания, на мой взгляд, кривая, как ятаган. Приведу отдельный пример:

In [ ]:
# Выведем информацию по двум тренерам: Бачину и Василеноку
# Ограничемся только ФИО и специализацией

coach_bachin = {'coach_name': re.sub(r'[^А-я, 0-9.-;:]', '', ''.join(tree.xpath('//table[@class="tcalendar"]/tbody/tr[6]/td[1]/text()'))),
                'specialization': re.sub(r'[^А-я, 0-9.-;:]', '',''.join(tree.xpath('//table[@class="tcalendar"]/tbody/tr[6]/td[5]/p/text()')))}
coach_vasilenok = {'coach_name': re.sub(r'[^А-я, 0-9.-;:]', '', ''.join(tree.xpath('//table[@class="tcalendar"]/tbody/tr[7]/td[1]/text()'))),
                'specialization': re.sub(r'[^А-я, 0-9.-;:]', '',''.join(tree.xpath('//table[@class="tcalendar"]/tbody/tr[7]/td[5]/p/text()')))}

print(coach_bachin)
print(coach_vasilenok)

{'coach_name': 'Бачин Валерий Петрович', 'specialization': 'все стили'}
{'coach_name': 'Василенок Алексей Викторович', 'specialization': ''}


Код отличается только индексами, в остальном все одинаково, НО!<br>
для Василенока в специализации пусто, хотя в таблице на сайте эта ячейка заполнена. Это значение можно получить следующим запросом:

In [ ]:
coach_vasilenok = {'coach_name': re.sub(r'[^А-я, 0-9.-;:]', '', ''.join(tree.xpath('//table[@class="tcalendar"]/tbody/tr[7]/td[1]/text()'))),
                'specialization': re.sub(r'[^А-я, 0-9.-;:]', '',''.join(tree.xpath('//table[@class="tcalendar"]/tbody/tr[7]/td[5]/text()')))}
print(coach_vasilenok)

{'coach_name': 'Василенок Алексей Викторович', 'specialization': 'вСЕ СТИЛИ'}


Отличие в тэге "p", что придает уникальность данному запросу, но из-за этого не получится прогнать его через цикл. Вопрос: как с этим быть?<br>
Может есть другой способ составления запроса через xpath с учетом таких "особенностей" html?

**Вопрос 2.**<br>
В таблице есть пустые значения. Рассмотрим это на примере тех же джентльменов:

In [ ]:
coaches = []
for i in range(6, 8):
  coaches.append({'coach_name': re.sub(r'[^А-я, 0-9.-;:]', '', ''.join(tree.xpath(f'//table[@class="tcalendar"]/tbody/tr[{i}]/td[1]/text()'))),
            'sport_rank_regalia': re.sub(r'[^А-я, 0-9.-;:]', '', ''.join(tree.xpath(f'//table[@class="tcalendar"]/tbody/tr[{i}]/td[2]/p/text()')))})

In [ ]:
df = pd.DataFrame(coaches)
df

,coach_name,sport_rank_regalia
0,Бачин Валерий Петрович,ЗТР
1,Василенок Алексей Викторович,


У Бачина колонка "Спорт. Звание, регалии" заполнена, а у Василенока нет. На семинаре вы говорили, что это приведет к ошибкам в анализе и искажению статистических данных. Собственно, вопрос тот же: как с этим быть? При работе с датафреймом заполнить пустые значения нулями, None или сделать что-то еще?

**Вопрос 3 (повышенной сложности).**<br>
Изначально я планировал работать с сайтом Olympteka.ru. Если посмотреть структуру сайта, то она очень простенькая. Но, как я не пытался, так и не смог извлечь ни одной ссылки.

In [ ]:
url = 'https://olympteka.ru/olymp/athlets/by_sport/3/1.html?md=gold'
resp = requests.get(url=url, headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'})
resp.status_code

200

In [ ]:
tree = html.fromstring(html=resp.content)

In [ ]:
# Попытка вывести список рядов из таблицы
sportsmen = tree.xpath('//table[@class="tb-eo tb-athlets tba-gold"]/tbody/tr')
sportsmen

[]

In [ ]:
# Попытка обратиться непосредственно к определенному элементу таблицы
athlete_name = tree.xpath('//table[@class="tb-eo tb-athlets tba-gold"]/tbody/tr[2]/td[1]/a/span/text()')
athlete_name

[]

Везде, как видите, получаются пустые списки. При этом неважно - задаешь ли отностительный путь или абсолютный с кучей div'ов, результат один.

Поэтому вопрос: что я делаю не так? Почему не удается выцепить ни одного значения?